In [1]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

/home/dic/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def conv_layer(input, channels_in, channels_out,name="conv"):
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        
        #w=tf.Variable(tf.truncated_normal([5,5,channels_in,channels_out], stddev=0.1),name="W")
        w=tf.get_variable('we', shape=(3,3,channels_in,channels_out), initializer=tf.contrib.layers.xavier_initializer()) 

        #initializer=tf.glorot_uniform_initializer()
        #,initializer=tf.truncated_normal_initializer(stddev=0.05)
      #  b=tf.Variable(tf.constant(0.1, shape=[channels_out]), name="B")
        b=tf.get_variable('be', shape=(channels_out), initializer=tf.contrib.layers.xavier_initializer())

        #,initializer=tf.random_uniform_initializer(-0.1,0.1)
        
        conv=tf.nn.conv2d(input,w, strides=[1,1,1,1], padding="SAME")
        act=tf.nn.relu(conv+b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

def dense_layer(input, channels_in, channels_out,name="dense"):
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        #w=tf.Variable(tf.truncated_normal([channels_in,channels_out], stddev=0.1),name="W")
        w=tf.get_variable('we1', shape=(channels_in,channels_out), initializer=tf.contrib.layers.xavier_initializer())
 
        #b=tf.Variable(tf.constant(0.1, shape=[channels_out]), name="B")
        b=tf.get_variable('be1', shape=(channels_out), initializer=tf.contrib.layers.xavier_initializer())

    
        act=tf.nn.relu(tf.matmul(input, w)+b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

In [3]:
#setup placeholders
x= tf.placeholder(tf.float32, shape=[None,28,28,1],name="x")#here None will take input at the time of training and it will take the value equal to the batch size 
y= tf.placeholder(tf.float32, shape=[None,7],name="labels")


#Creating the network

#def conv_net(x):
conv1=conv_layer(x, 1, 32,"conv1")
drop1=tf.nn.dropout(conv1,keep_prob=0.1,name="dropout1")
flattened = tf.reshape(drop1, [-1, 28 * 28 * 32])

dense1=dense_layer(flattened,28*28*32,128,"dense1")
drop2=tf.nn.dropout(dense1,keep_prob=0.25,name="dropout2")
dense2=dense_layer(drop2,128,128,"dense2")
logits=dense_layer(dense2,128,7,"dense3")



In [4]:
#pred = conv_net(x)

#cross entropy as loss function
#cost=tf.reduce_mean=(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y))
with tf.name_scope("Loss"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    tf.summary.scalar("Loss", cost)
#adamoptimizer
#optimizer=tf.train.AdamOptimizer(0.0002).minimize(cost)
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(cost)
    
#computing accuracy
with tf.name_scope("accuracy"):
#Here you check whether the index of the maximum value of the predicted image is equal to the actual labelled image. and both will be a column vector.
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))

    #calculate accuracy across all the given images and average them out. 
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("Accuracy", accuracy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [5]:
import numpy as np
#import tensorflow.contrib.eager as tfe
# Load the training data into two NumPy arrays.

classes=7
features = np.load("X_2d.npy")
labels = np.load("y_onehot.npy")
X_train1, X_test1, y_train1, y_test1 = train_test_split(features, labels, test_size=0.2, random_state=1, shuffle=True)
print(y_train1.shape)
X_train = X_train1.reshape(-1, 28, 28, 1)
X_test= X_test1.reshape(-1,28,28,1)
y_train=y_train1.reshape(-1,7)
y_test=y_test1.reshape(-1,7)
print(X_train.shape)
print(y_train)
print(y_train.shape)


epoch=700
batch_size=128
steps=int(len(X_train)/batch_size)

(36504, 7)
(36504, 28, 28, 1)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
(36504, 7)


In [6]:

sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
writer_1 = tf.summary.FileWriter("/home/dic/jupyter/28x28dmdt/log_dir/train_1_700epochs_with_lr_0.0002_xavier_weights_1")
#writer_2 = tf.summary.FileWriter("/home/dic/jupyter/28x28dmdt/log_dir/test_1_200epochs")
writer_1.add_graph(sess.graph)
write_op = tf.summary.merge_all()

In [8]:

train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []
#writer = tf.summary.FileWriter('/home/dic/jupyter/28x28dmdt/log_dir/1', sess.graph)    
    
for e in range(epoch):
    epoch_loss=0
    
    for batch in range(steps):
        #Calculate our current step
        step = e    *steps + batch
    
        batch_x = X_train[batch*batch_size:min((batch+1)*batch_size,len(X_train))]
        batch_y = y_train[batch*batch_size:min((batch+1)*batch_size,len(y_train))]
        
            #batch,c=sess.run([optimizer,loss],feed_dict={x:batch_x,y=batch_y})
        
        opt = sess.run(optimizer, feed_dict={x: batch_x,y: batch_y})
        loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,y: batch_y})
        if step % 100 == 0:
            
            # Get Train Summary for one batch and add summary to TensorBoard                                                      
            summary = sess.run(write_op,feed_dict={x: batch_x,y: batch_y})
            writer_1.add_summary(summary, step)
            writer_1.flush()
    
    print("Iter " + str(e) + ", Loss= ",loss , ", Training Accuracy= ",acc)                                     
    print("Optimization Finished!")
        
       
    test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={x: X_test,y : y_test})
    
    train_loss.append(loss)
    test_loss.append(valid_loss)
    train_accuracy.append(acc)
    test_accuracy.append(test_acc)
    #summary1 = sess.run(write_op,feed_dict={x: X_test,y : y_test})
    #writer_2.add_summary(summary1)
    #writer_2.flush()
    print("Testing Accuracy:",test_acc)
        #print("Testing Accuracy:","{:.5f}".format(test_acc))
        

Iter 0, Loss=  1.1129481 , Training Accuracy=  0.6328125
Optimization Finished!
Testing Accuracy: 0.66239315
Iter 1, Loss=  1.0269735 , Training Accuracy=  0.640625
Optimization Finished!
Testing Accuracy: 0.6710498
Iter 2, Loss=  1.0487123 , Training Accuracy=  0.65625
Optimization Finished!
Testing Accuracy: 0.68091166
Iter 3, Loss=  1.0008156 , Training Accuracy=  0.65625
Optimization Finished!
Testing Accuracy: 0.68858206
Iter 4, Loss=  0.95470566 , Training Accuracy=  0.6875
Optimization Finished!
Testing Accuracy: 0.7009643
Iter 5, Loss=  0.89413214 , Training Accuracy=  0.6796875
Optimization Finished!
Testing Accuracy: 0.7045803
Iter 6, Loss=  0.82510257 , Training Accuracy=  0.703125
Optimization Finished!
Testing Accuracy: 0.71290815
Iter 7, Loss=  0.81000787 , Training Accuracy=  0.7265625
Optimization Finished!
Testing Accuracy: 0.7110454
Iter 8, Loss=  0.8536639 , Training Accuracy=  0.6796875
Optimization Finished!
Testing Accuracy: 0.72452337
Iter 9, Loss=  0.8268359 , T

Testing Accuracy: 0.78402364
Iter 76, Loss=  0.61964047 , Training Accuracy=  0.7890625
Optimization Finished!
Testing Accuracy: 0.77854484
Iter 77, Loss=  0.6395469 , Training Accuracy=  0.7421875
Optimization Finished!
Testing Accuracy: 0.7760245
Iter 78, Loss=  0.5789831 , Training Accuracy=  0.796875
Optimization Finished!
Testing Accuracy: 0.78402364
Iter 79, Loss=  0.5936549 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.78347576
Iter 80, Loss=  0.5830002 , Training Accuracy=  0.7734375
Optimization Finished!
Testing Accuracy: 0.7805172
Iter 81, Loss=  0.56547844 , Training Accuracy=  0.7890625
Optimization Finished!
Testing Accuracy: 0.7815034
Iter 82, Loss=  0.6075995 , Training Accuracy=  0.7890625
Optimization Finished!
Testing Accuracy: 0.7815034
Iter 83, Loss=  0.66375643 , Training Accuracy=  0.8046875
Optimization Finished!
Testing Accuracy: 0.7767916
Iter 84, Loss=  0.5866902 , Training Accuracy=  0.7578125
Optimization Finished!
Testing Accuracy

Testing Accuracy: 0.7947622
Iter 151, Loss=  0.51393497 , Training Accuracy=  0.8515625
Optimization Finished!
Testing Accuracy: 0.796625
Iter 152, Loss=  0.56451994 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.7942143
Iter 153, Loss=  0.6025708 , Training Accuracy=  0.796875
Optimization Finished!
Testing Accuracy: 0.7982687
Iter 154, Loss=  0.5707672 , Training Accuracy=  0.78125
Optimization Finished!
Testing Accuracy: 0.79355687
Iter 155, Loss=  0.46823943 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.79377604
Iter 156, Loss=  0.4903428 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.796625
Iter 157, Loss=  0.58071774 , Training Accuracy=  0.796875
Optimization Finished!
Testing Accuracy: 0.7954197
Iter 158, Loss=  0.6100864 , Training Accuracy=  0.765625
Optimization Finished!
Testing Accuracy: 0.7968442
Iter 159, Loss=  0.5228617 , Training Accuracy=  0.8359375
Optimization Finished!
Testing Accura

Testing Accuracy: 0.804405
Iter 226, Loss=  0.5408704 , Training Accuracy=  0.84375
Optimization Finished!
Testing Accuracy: 0.8050625
Iter 227, Loss=  0.53527355 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.80111766
Iter 228, Loss=  0.49690852 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.7995836
Iter 229, Loss=  0.52193695 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.8062678
Iter 230, Loss=  0.57716465 , Training Accuracy=  0.8046875
Optimization Finished!
Testing Accuracy: 0.804405
Iter 231, Loss=  0.48518983 , Training Accuracy=  0.84375
Optimization Finished!
Testing Accuracy: 0.8048433
Iter 232, Loss=  0.60312533 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.7995836
Iter 233, Loss=  0.52948666 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.8030901
Iter 234, Loss=  0.51880324 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accur

Testing Accuracy: 0.8082402
Iter 301, Loss=  0.5421032 , Training Accuracy=  0.796875
Optimization Finished!
Testing Accuracy: 0.8113083
Iter 302, Loss=  0.56046027 , Training Accuracy=  0.796875
Optimization Finished!
Testing Accuracy: 0.80374753
Iter 303, Loss=  0.5257612 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.80988383
Iter 304, Loss=  0.5360125 , Training Accuracy=  0.78125
Optimization Finished!
Testing Accuracy: 0.80802107
Iter 305, Loss=  0.46228802 , Training Accuracy=  0.828125
Optimization Finished!
Testing Accuracy: 0.8116371
Iter 306, Loss=  0.55636543 , Training Accuracy=  0.828125
Optimization Finished!
Testing Accuracy: 0.80758274
Iter 307, Loss=  0.5426669 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.8090072
Iter 308, Loss=  0.53183603 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.80999345
Iter 309, Loss=  0.589259 , Training Accuracy=  0.796875
Optimization Finished!
Testing Accu

Testing Accuracy: 0.81076044
Iter 376, Loss=  0.44864964 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.8125137
Iter 377, Loss=  0.47959697 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.80988383
Iter 378, Loss=  0.5259359 , Training Accuracy=  0.828125
Optimization Finished!
Testing Accuracy: 0.81437653
Iter 379, Loss=  0.450297 , Training Accuracy=  0.828125
Optimization Finished!
Testing Accuracy: 0.81054133
Iter 380, Loss=  0.47957736 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.80988383
Iter 381, Loss=  0.47807023 , Training Accuracy=  0.8359375
Optimization Finished!
Testing Accuracy: 0.8136095
Iter 382, Loss=  0.47467136 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.8086785
Iter 383, Loss=  0.4643308 , Training Accuracy=  0.828125
Optimization Finished!
Testing Accuracy: 0.8136095
Iter 384, Loss=  0.4435593 , Training Accuracy=  0.859375
Optimization Finished!
Testing 

Testing Accuracy: 0.8115275
Iter 451, Loss=  0.4508229 , Training Accuracy=  0.828125
Optimization Finished!
Testing Accuracy: 0.80999345
Iter 452, Loss=  0.584655 , Training Accuracy=  0.78125
Optimization Finished!
Testing Accuracy: 0.8142669
Iter 453, Loss=  0.49416003 , Training Accuracy=  0.828125
Optimization Finished!
Testing Accuracy: 0.8165681
Iter 454, Loss=  0.5594853 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.81141794
Iter 455, Loss=  0.4685434 , Training Accuracy=  0.828125
Optimization Finished!
Testing Accuracy: 0.8116371
Iter 456, Loss=  0.45583674 , Training Accuracy=  0.796875
Optimization Finished!
Testing Accuracy: 0.81415737
Iter 457, Loss=  0.5268184 , Training Accuracy=  0.7890625
Optimization Finished!
Testing Accuracy: 0.81317115
Iter 458, Loss=  0.4978091 , Training Accuracy=  0.828125
Optimization Finished!
Testing Accuracy: 0.81415737
Iter 459, Loss=  0.49217606 , Training Accuracy=  0.84375
Optimization Finished!
Testing Accurac

Testing Accuracy: 0.8118562
Iter 526, Loss=  0.41908026 , Training Accuracy=  0.8671875
Optimization Finished!
Testing Accuracy: 0.81755424
Iter 527, Loss=  0.5077928 , Training Accuracy=  0.8359375
Optimization Finished!
Testing Accuracy: 0.8196362
Iter 528, Loss=  0.43655354 , Training Accuracy=  0.8515625
Optimization Finished!
Testing Accuracy: 0.8149244
Iter 529, Loss=  0.50210285 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.81645846
Iter 530, Loss=  0.48627865 , Training Accuracy=  0.84375
Optimization Finished!
Testing Accuracy: 0.8176638
Iter 531, Loss=  0.46527457 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.81262326
Iter 532, Loss=  0.46723074 , Training Accuracy=  0.8515625
Optimization Finished!
Testing Accuracy: 0.8188692
Iter 533, Loss=  0.43140543 , Training Accuracy=  0.8671875
Optimization Finished!
Testing Accuracy: 0.815034
Iter 534, Loss=  0.46011624 , Training Accuracy=  0.84375
Optimization Finished!
Testing

Testing Accuracy: 0.8195266
Iter 601, Loss=  0.47250777 , Training Accuracy=  0.796875
Optimization Finished!
Testing Accuracy: 0.81558186
Iter 602, Loss=  0.44459468 , Training Accuracy=  0.84375
Optimization Finished!
Testing Accuracy: 0.8119658
Iter 603, Loss=  0.41303897 , Training Accuracy=  0.859375
Optimization Finished!
Testing Accuracy: 0.8212799
Iter 604, Loss=  0.43697783 , Training Accuracy=  0.84375
Optimization Finished!
Testing Accuracy: 0.8171159
Iter 605, Loss=  0.42199808 , Training Accuracy=  0.84375
Optimization Finished!
Testing Accuracy: 0.8167872
Iter 606, Loss=  0.3962991 , Training Accuracy=  0.8515625
Optimization Finished!
Testing Accuracy: 0.81021255
Iter 607, Loss=  0.44302443 , Training Accuracy=  0.90625
Optimization Finished!
Testing Accuracy: 0.81415737
Iter 608, Loss=  0.49943694 , Training Accuracy=  0.859375
Optimization Finished!
Testing Accuracy: 0.81371903
Iter 609, Loss=  0.40467337 , Training Accuracy=  0.8359375
Optimization Finished!
Testing A

Testing Accuracy: 0.8159106
Iter 676, Loss=  0.44626397 , Training Accuracy=  0.8515625
Optimization Finished!
Testing Accuracy: 0.8133903
Iter 677, Loss=  0.48004916 , Training Accuracy=  0.8515625
Optimization Finished!
Testing Accuracy: 0.8182117
Iter 678, Loss=  0.47459486 , Training Accuracy=  0.8359375
Optimization Finished!
Testing Accuracy: 0.81349987
Iter 679, Loss=  0.4484858 , Training Accuracy=  0.8515625
Optimization Finished!
Testing Accuracy: 0.81700635
Iter 680, Loss=  0.47273183 , Training Accuracy=  0.8359375
Optimization Finished!
Testing Accuracy: 0.81941706
Iter 681, Loss=  0.54209757 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.8151435
Iter 682, Loss=  0.4646191 , Training Accuracy=  0.8515625
Optimization Finished!
Testing Accuracy: 0.81996495
Iter 683, Loss=  0.43528432 , Training Accuracy=  0.859375
Optimization Finished!
Testing Accuracy: 0.81459564
Iter 684, Loss=  0.40165615 , Training Accuracy=  0.859375
Optimization Finished!
Tes